In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import accuracy
from surprise import dump
from surprise.model_selection import train_test_split

from utilities import data_basic_utility as databasic
from utilities import dataframe_utility as dfutil
import features_utility as featutil

## File Details

Working notebook of a Full Ensemble Run structure, with just KNNWithMeans Model


In [ ]:
filePrefix = "A3_092_ensemble_v2_complete_run_200k"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
runDir = "runs/"
modelsDir = "models/"
featuresDataDir = "features/"

seed = databasic.get_random_seed()

In [ ]:
# trainFilePath = baseDataDir + 'train.tsv'
# valiFilePath = baseDataDir + 'val.tsv'
# featuresFilePath = baseDataDir + 'features.tsv'
# testFilePath = baseDataDir + 'test.tsv'

trainFilePath = baseDataDir + 'train_200k.tsv'
valiFilePath = baseDataDir + 'vali_200k.tsv'
featuresFilePath = baseDataDir + 'features_200k.tsv'
testFilePath = baseDataDir + 'test_200k.tsv'

Load the Files one by one then delete them after your done, for memory management

In [ ]:
df_train = pd.read_csv(trainFilePath,sep='\t',
              names=['RowID','BeerID','ReviewerID','BeerName','BeerType','rating'])
df_train.head(10)

In [ ]:
# Setup the data to be just the Reviewer and the Beer(Item) and the Rating Label we want to learn.
dfTrainFeatures = df_train.drop(['RowID','BeerName','BeerType'],axis=1)

## Collaborative Filter Models: Train

For the Collaborative Filtering Models, we only need the Training set. Train the models, then save them to file for later use

In [ ]:
# Load into a Surprise dataset
reader = Reader(rating_scale=(0, 5))
dsetTrainFeatures = Dataset.load_from_df(dfTrainFeatures[['BeerID','ReviewerID', 'rating']],reader)
trainsetTrainFeatures = dsetTrainFeatures.build_full_trainset()

In [ ]:
def trainSurpriseModel(algorithm, trainset, modelsDir, filePrefix, modelName):
  # Train the model then Save the predictor model to file
  model = algorithm.fit(trainset)  
  dump.dump(modelsDir + filePrefix + "_" + modelName + "_predictor.model", None, model, True)

In [ ]:
# Create each algorithm, train the model, save it to file for later, then delete the model

predictorKNN = KNNWithMeans(k=80)
trainSurpriseModel(predictorKNN, trainsetTrainFeatures, modelsDir, filePrefix, "knnwithmeans")
del predictorKNN

# predictorBaselineOnly = BaselineOnly(bsl_options = {'n_epochs': 8, 'reg_u': 4, 'reg_i': 15})
# trainSurpriseModel(predictorBaselineOnly, trainsetTrainFeatures, modelsDir, filePrefix, "baselineonly")
# del predictorBaselineOnly

# predictorSVDpp = SVDpp(n_factors = 10, n_epochs=20, lr_all=0.005, reg_all=0.2)
# trainSurpriseModel(predictorSVDpp, trainsetTrainFeatures, modelsDir, filePrefix, "svdpp")
# del predictorSVDpp

In [ ]:
# Clean up the training data used for the collaborate filters
del trainsetTrainFeatures
del reader
del dsetTrainFeatures
del dfTrainFeatures
del df_train

## Collaborative Filter Models: Predict On Validation Data 

Now we want to load the Validation set to we can predict against it and write out the subrun files, which will be used later for the Ensemble.

First, do the Predictions for the Collaborative Filter models (surprise)

In [ ]:
# Read the validation data (in full)
df_vali = pd.read_csv(valiFilePath,sep='\t',
              names=['RowID','BeerID','ReviewerID','BeerName','BeerType','rating'])

reader = Reader(rating_scale=(0, 5))

idCols = ['RowID','BeerID','ReviewerID']
dfValiIds = df_vali[idCols]
dfValiFeatures = df_vali.drop(['RowID','BeerName','BeerType'],axis=1)

dsetValiFeatures = Dataset.load_from_df(dfValiFeatures[['BeerID','ReviewerID', 'rating']],reader)

In [ ]:
def predictSurpriseModel(modelsDir, filePrefix, modelName, dsName, dataset, dfIds, subrunDir):
  # Load the algorithm from the file, the predictions aren't used so that variable will be None
  predictions, algorithm = dump.load(modelsDir + filePrefix + "_" + modelName + "_predictor.model")
  
  # Make Predictions using the model
  NA,valset = train_test_split(dataset, test_size=1.0)
  predictions = algorithm.test(valset)
  
  # Display the MAE
  mae = accuracy.mae(predictions,verbose=True)
  print("MAE for " + modelName + ": " + str(mae))

  # Convert the Predictions to a dataframe so we can lookup predictions easy
  # uid == BeerId, iid == ReviewerId, r_ui == Original Ration, est = Predicted rating
  lstUIds = list(map(lambda x: x.uid, predictions))
  lstIIds = list(map(lambda x: x.iid, predictions))
  lstTrueRatings = list(map(lambda x: x.r_ui, predictions))
  lstRatingEst = list(map(lambda x: x.est, predictions))
  dfPredictions = pd.DataFrame({ "uid": lstUIds,"iid": lstIIds, "r_ui": lstTrueRatings, "Predict": lstRatingEst })  

  # join the predictions to the ids, sort by rowid and write to out the subrun file
  subRunFilePath = subrunDir + filePrefix + "_" + modelName + "_" + dsName + "_subrun.csv"
  dfPredictions = pd.merge(dfIds, dfPredictions, how="inner", left_on=["BeerID", "ReviewerID"], right_on=["uid", "iid"])
  dfPredictions.sort_values("RowID")[["RowID", "BeerID", "ReviewerID", "Predict"]].to_csv(subRunFilePath, index=False)

  # Clean up the variables from memory
  del predictions
  del algorithm
  del valset
  del lstUIds
  del lstIIds
  del lstTrueRatings
  del lstRatingEst
  del dfPredictions


In [ ]:
predictSurpriseModel(modelsDir, filePrefix, "knnwithmeans", "val", dsetValiFeatures, dfValiIds, subrunDir)
# predictSurpriseModel(modelsDir, filePrefix, "baselineonly", "val", dsetValiFeatures, dfValiIds, subrunDir)
# predictSurpriseModel(modelsDir, filePrefix, "svdpp", "val", dsetValiFeatures, dfValiIds, subrunDir)

In [ ]:
# Clean up variables from the Predict Stage
del df_vali
del reader
del dfValiIds
del dfValiFeatures
del dsetValiFeatures

## Content Filter Models, train and predict

First we want to load the features and do all the data preprocessing, then we can train the different models

In [ ]:
# Load the training data
df_train = pd.read_csv(trainFilePath,sep='\t',
              names=['RowID','BeerID','ReviewerID','BeerName','BeerType','rating'])

# Load the validation data. When we want to do one hot encoding, we have to do it over both datasets to ensure consistency
df_vali = pd.read_csv(valiFilePath,sep='\t',
                         names=['RowID','BeerID','ReviewerID', 'BeerName','BeerType','rating'])

# Load the validation data. When we want to do one hot encoding, we have to do it over both datasets to ensure consistency
df_test = pd.read_csv(testFilePath,sep='\t',
                         names=['RowID','BeerID','ReviewerID', 'BeerName','BeerType','rating'])                         

# Load the features
df_features = pd.read_csv(featuresFilePath,sep='\t',
    names=['RowID','BrewerID','ABV','DayofWeek','Month',
          'DayofMonth','Year','TimeOfDay','Gender',
          'Birthday','Text','Lemmatized','POS_Tag'])    

In [ ]:
# Add the reviewer counts to each of the data sets
df_train = featutil.addReviewerReviewCount(df_train)
df_train = featutil.addBeerReviewCount(df_train)

df_vali = featutil.addReviewerReviewCount(df_vali)
df_vali = featutil.addBeerReviewCount(df_vali)

df_test = featutil.addReviewerReviewCount(df_test)
df_test = featutil.addBeerReviewCount(df_test)

In [ ]:
colsToUse = ["RowID", "BrewerID", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender", "TimeOfDay"]

df_train_data = df_train.join(df_features[colsToUse], on="RowID", how="inner", rsuffix="Feat")
df_vali_data = df_vali.join(df_features[colsToUse], on="RowID", how="inner", rsuffix="Feat")
df_test_data = df_test.join(df_features[colsToUse], on="RowID", how="inner", rsuffix="Feat")

# Remove the duplicated Row ID, also remove Beer Name at this point, we're nt using it
df_train_data = df_train_data.drop(['RowIDFeat', "BeerName"],axis=1)
df_vali_data = df_vali_data.drop(['RowIDFeat', "BeerName"],axis=1)
df_test_data = df_test_data.drop(['RowIDFeat', "BeerName"],axis=1)

In [ ]:
# Clean up these dataframes now that they have been joined
del df_train
del df_vali
del df_features
del df_test

In [ ]:
# do the feature transformations
df_train_data = featutil.fixNullABV(df_train_data)
df_vali_data = featutil.fixNullABV(df_vali_data)
df_test_data = featutil.fixNullABV(df_test_data)

df_train_data, df_vali_data, df_test_data = dfutil.getDummiesForTripleSets(df_train_data, df_vali_data, df_test_data, "BrewerID")

df_train_data, df_vali_data, df_test_data = dfutil.getDummiesForTripleSets(df_train_data, df_vali_data, df_test_data, "BeerType")

df_train_data, df_vali_data, df_test_data = dfutil.getDummiesForTripleSets(df_train_data, df_vali_data, df_test_data, "Gender")

df_train_data = featutil.formatDayOfWeek(df_train_data)
df_vali_data = featutil.formatDayOfWeek(df_vali_data)
df_test_data = featutil.formatDayOfWeek(df_test_data)

df_train_data = featutil.formatMonth(df_train_data)
df_vali_data = featutil.formatMonth(df_vali_data)
df_test_data = featutil.formatMonth(df_test_data)

df_train_data = featutil.formatTimeToSec(df_train_data)
df_vali_data = featutil.formatTimeToSec(df_vali_data)
df_test_data = featutil.formatTimeToSec(df_test_data)

In [ ]:
print(df_train_data.shape)
print(df_vali_data.shape)
print(df_test_data.shape)
df_test_data.head()

In [ ]:
# Write the test data file out so we can load it back in later so as not to have to redo this step
df_test_data.to_csv(featuresDataDir + filePrefix + "_test_cleaned.csv", index=False)

In [ ]:
# Get all the columns
col_names = df_train_data.columns

idCols = ['RowID','BeerID','ReviewerID']
feature_cols =  col_names.drop(['RowID','BeerID','ReviewerID','rating' ])
target_col = 'rating'

# Create the sub data sets of the features and the target
dfTrainFeatures = df_train_data[feature_cols]
dfTrainTarget = df_train_data[target_col]

dfValiIds = df_vali_data[idCols]
dfValiFeatures = df_vali_data[feature_cols]
dfValiTarget = df_vali_data[target_col]


In [ ]:
def trainLightGbmModel(model, train_feat, train_target, 
      modelsDir, filePrefix, modelName):

  # Train the model and Save the predictor model to file
  model.fit(X=train_feat, y=train_target) 
  model.booster_.save_model(modelsDir + filePrefix + "_" + modelName + "_predictor.model")


def trainSkLinearRegModel(model, 
      train_feat, train_target, vali_ids, vali_feat, vali_target, 
      modelsDir, filePrefix, dsName, modelName):
  # Train the model then Save the predictor model to file
  model.fit(train_feat, train_target) 
  pickle.dump(model, open(modelsDir + filePrefix + "_" + modelName + "_predictor.model", 'wb'))

In [ ]:
def predictLightGbmModel(vali_ids, vali_feat, vali_target, 
      modelsDir, filePrefix, dsName, modelName):

  model = lgb.Booster(model_file=modelsDir + filePrefix + "_" + modelName + "_predictor.model")

  predicted = model.predict(vali_feat)
  dfPredicted = pd.DataFrame({"Predict": predicted})

  # join the predictions to the ids, sort by rowid and write to out the subrun file
  subRunFilePath = subrunDir + filePrefix + "_" + modelName + "_" + dsName + "_subrun.csv"
  dfPredicted = pd.concat([vali_ids.reset_index(), dfPredicted], axis=1).drop(columns="index")
  dfPredicted.to_csv(subRunFilePath, index=False)

  if vali_target is not None:      
    mae = mean_absolute_error(vali_target, predicted)
    print("MAE for " + modelName + ": " + str(mae))

  # clean up variables in memory
  del predicted
  del dfPredicted


def predictSkLinearRegModel(vali_ids, vali_feat, vali_target, 
      modelsDir, filePrefix, dsName, modelName):

  model = pickle.load(open(modelsDir + filePrefix + "_" + modelName + "_predictor.model", 'rb'))

  predicted = model.predict(vali_feat)
  dfPredicted = pd.DataFrame({"Predict": predicted})

  # join the predictions to the ids, sort by rowid and write to out the subrun file
  subRunFilePath = subrunDir + filePrefix + "_" + modelName + "_" + dsName + "_subrun.csv"
  dfPredicted = pd.concat([vali_ids.reset_index(), dfPredicted], axis=1).drop(columns="index")
  dfPredicted.to_csv(subRunFilePath, index=False)

  # When used on the test data, we have not target/label data. This is just used for evaluation, to 
  # calculate our MAE against real labels. If none is passed in, don't do this
  if vali_target is not None:      
    mae = mean_absolute_error(vali_target, predicted)
    print("MAE for " + modelName + ": " + str(mae))

  # clean up variables in memory
  del predicted
  del dfPredicted  


In [ ]:
dfTrainFeatures.head()

In [ ]:
def getFeaturesBeerContext(df1):
  consumerCols = ["DayofWeek", "DayofMonth", "Month", "TimeOfDay", "Gender_Male", "Gender_Female", "Gender_unknown"]
  return dfutil.getFeaturesWithoutCols(df1, consumerCols)

In [ ]:
# Train the models, save them to file and then clear the model from memory
modelBeerContext = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed
    ,learning_rate=0.010443500090385492, num_leaves = 68, max_depth = 14, n_estimators = 608
  )  
dfTrainFeatures_BeerContext = getFeaturesBeerContext(dfTrainFeatures)
dfValiFeatures_BeerContext = getFeaturesBeerContext(dfValiFeatures)
trainLightGbmModel(modelBeerContext, dfTrainFeatures_BeerContext, dfTrainTarget, 
    modelsDir, filePrefix, "lgbm_beercontext")
predictLightGbmModel(dfValiIds, dfValiFeatures_BeerContext, dfValiTarget,
    modelsDir, filePrefix, "val", "lgbm_beercontext")    
del dfTrainFeatures_BeerContext
del dfValiFeatures_BeerContext
del modelBeerContext


# modelConsumerContext = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed
#     ,learning_rate=0.26879548049242075, num_leaves = 91, max_depth = 2, n_estimators = 384
#  ) 
# dfTrainFeatures_ConsumerContext = dfTrainFeatures[["Year", "ReviewerReviewCount", "BeerReviewCount", "DayofWeek", "DayofMonth", "Month", "TimeOfDay", "Gender_Male", "Gender_Female", "Gender_unknown"]]
# dfValiFeatures_ConsumerContext = dfValiFeatures[["Year", "ReviewerReviewCount", "BeerReviewCount", "DayofWeek", "DayofMonth", "Month", "TimeOfDay", "Gender_Male", "Gender_Female", "Gender_unknown"]]
# trainLightGbmModel(modelConsumerContext, dfTrainFeatures_ConsumerContext, dfTrainTarget, 
#     modelsDir, filePrefix, "lgbm_consumercontext")
# predictLightGbmModel(dfValiIds, dfValiFeatures_ConsumerContext, dfValiTarget,
#     modelsDir, filePrefix, "val", "lgbm_consumercontext")    
# del dfTrainFeatures_ConsumerContext
# del dfValiFeatures_ConsumerContext
# del modelConsumerContext


# modelLinReg = LinearRegression()
# trainSkLinearRegModel(modelLinReg, dfTrainFeatures, dfTrainTarget, 
#     modelsDir, filePrefix, "val", "sklinearreg")
# predictSkLinearRegModel(dfValiIds, dfValiFeatures, dfValiTarget,
#     modelsDir, filePrefix, "val", "sklinearreg")
# del modelLinReg

In [ ]:
# Clean up the variables from memory
del df_train_data
del df_vali_data
del df_test_data
del dfTrainFeatures
del dfTrainTarget
del dfValiIds
del dfValiFeatures
del dfValiTarget

### Train the Ensemble Model

Now that all the sub run files have been generated, combine all the predictions into one dataset, train a new final, ensemble model, predict on the validation data and get an MAE and save the model for use later on the Test data.

In [ ]:
# Read the validation data (in full) again. But this time, we just want the Row and the rating
df_vali = pd.read_csv(valiFilePath,sep='\t',
              names=['RowID','BeerID','ReviewerID','BeerName','BeerType','rating'])

df_ensemble_full = df_vali[["RowID", "rating"]]      

del df_vali

In [ ]:
# Load all the sub runs and join them together with the ensemble data

# Collaborative Filter Runs
fileName = filePrefix + "_" + "knnwithmeans" + "_val" + "_subrun"
df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

# fileName = filePrefix + "_" + "baselineonly" + "_val" + "_subrun"
# df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

# fileName = filePrefix + "_" + "svdpp" + "_val" + "_subrun"
# df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

# # Content Filter Runs
fileName = filePrefix + "_" + "lgbm_beercontext" + "_val" + "_subrun"
df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

# fileName = filePrefix + "_" + "lgbm_consumercontext" + "_val" + "_subrun"
# df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

# fileName = filePrefix + "_" + "sklinearreg" + "_val" + "_subrun"
# df_ensemble_full = featutil.joinRunToEnsembleFrame(df_ensemble_full, subrunDir, fileName)

In [ ]:
# Get all the columns
col_names = df_ensemble_full.columns

idCols = ['RowID']
feature_cols =  col_names.drop(['RowID','rating'])
target_col = 'rating'

# Create the sub data sets of the features and the target
dfTrainIds = df_ensemble_full[idCols]
dfTrainFeatures = df_ensemble_full[feature_cols]
dfTrainTarget = df_ensemble_full[target_col]


In [ ]:
# Doing the final Ensemble prediction using Light GBM Regression, params tuned

# Create the model and predict
model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed,
  learning_rate=0.298864877137463, num_leaves=127, max_depth=26, n_estimators=974
)
model.fit(X=dfTrainFeatures, y=dfTrainTarget)

# use the model to predict
test_predicted = model.predict(dfTrainFeatures)
dfPredicted = pd.DataFrame({"Predict": test_predicted})

# Calc the MAE and display
mae = mean_absolute_error(dfTrainTarget, test_predicted)
print("Ensemble Final Average MAE (from validation data): " + str(mae))

# Save the model to file
model.booster_.save_model(modelsDir + filePrefix + "_ensemble_predictor.model")

In [ ]:
# Clean up all the variables
del df_ensemble_full
del dfTrainIds
del dfTrainFeatures
del dfTrainTarget
del model
del test_predicted
del dfPredicted

## Predict on the Test data with Models for Subruns

Now that we have the final Ensemble model, we can process the Test data. We need to load the test data, and create all the sub runs by using all the base level models to predict.

First, predict using the Collaborative Filter Models

In [ ]:
# Read the validation data (in full)
df_test = pd.read_csv(testFilePath,sep='\t',
              names=['RowID','BeerID','ReviewerID','BeerName','BeerType'])

# The test set is unlabeled, so we don't know the true ratings. Populate a rating col with zeros, as we are going
# to predict these values
df_test["rating"] = 0

reader = Reader(rating_scale=(0, 5))

idCols = ['RowID','BeerID','ReviewerID']
dfTestIds = df_test[idCols]
dfTestFeatures = df_test.drop(['RowID','BeerName','BeerType'],axis=1)

dsetTestFeatures = Dataset.load_from_df(dfTestFeatures[['BeerID','ReviewerID','rating']],reader)

In [ ]:
# Predict using the Collaborative Filter Models
predictSurpriseModel(modelsDir, filePrefix, "knnwithmeans", "test", dsetTestFeatures, dfTestIds, subrunDir)
# predictSurpriseModel(modelsDir, filePrefix, "baselineonly", "test", dsetTestFeatures, dfTestIds, subrunDir)
# predictSurpriseModel(modelsDir, filePrefix, "svdpp", "test", dsetTestFeatures, dfTestIds, subrunDir)

In [ ]:
# Clean up variables from the Predict Stage
del reader
del dfTestIds
del dfTestFeatures
del dsetTestFeatures

# Keep this, as we will use this in the next stage
# del df_test 

Now Predict using the Content Filter Models. 

In [ ]:
# Reload that test data that was cleaned and processed previously
df_test_data = pd.read_csv(featuresDataDir + filePrefix + "_test_cleaned.csv")


In [ ]:
# colsToUse = ["RowID", "BrewerID", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender", "TimeOfDay"]


In [ ]:
df_test_data.head()

In [ ]:
# Get all the columns
col_names = df_test_data.columns

idCols = ['RowID','BeerID','ReviewerID']
feature_cols =  col_names.drop(['RowID','BeerID','ReviewerID', 'rating' ])

# Create the sub data sets of the features and the target
dfTestIds = df_test_data[idCols]
dfTestFeatures = df_test_data[feature_cols]

In [ ]:
print(df_test_data.columns)
df_test_data.head()

Problem is with one hot encoding, different sets of brewers or beer types between the training data (train+vali) and what is in test

In [ ]:
# Now we can make predictions according to each of our Content Filter Models. Pass None for the target set, the function
# will just skip the evaluation (calculating the MAE)
dfTestFeatures_BeerContext =  getFeaturesBeerContext(dfTestFeatures)
predictLightGbmModel(dfTestIds, dfTestFeatures_BeerContext, None,
    modelsDir, filePrefix, "test", "lgbm_beercontext")    
del dfTestFeatures_BeerContext


# dfTestFeatures_ConsumerContext = dfTestFeatures[["Year", "ReviewerReviewCount", "BeerReviewCount", "DayofWeek", "DayofMonth", "Month", "TimeOfDay", "Gender_Male", "Gender_Female", "Gender_unknown"]]
# predictLightGbmModel(dfTestIds, dfTestFeatures_ConsumerContext, None,
#     modelsDir, filePrefix, "test", "lgbm_consumercontext") 
# del dfTestFeatures_ConsumerContext   


# modelLinReg = LinearRegression()
# predictSkLinearRegModel(dfTestIds, dfValiFeatures, None,
#     modelsDir, filePrefix, "test", "sklinearreg")
# del modelLinReg

### Load the Ensemble Model and predict on the Test data

Load the test data

In [ ]:
df_ensemble_test = df_test[["RowID"]]      

del df_test

In [ ]:
# Load all the sub runs and join them together with the ensemble data

# Collaborative Filter Runs
fileName = filePrefix + "_" + "knnwithmeans" + "_test" + "_subrun"
df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

# fileName = filePrefix + "_" + "baselineonly" + "_test" + "_subrun"
# df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

# fileName = filePrefix + "_" + "svdpp" + "_test" + "_subrun"
# df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

# # Content Filter Runs
fileName = filePrefix + "_" + "lgbm_beercontext" + "_test" + "_subrun"
df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

# fileName = filePrefix + "_" + "lgbm_consumercontext" + "_test" + "_subrun"
# df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

# fileName = filePrefix + "_" + "sklinearreg" + "_test" + "_subrun"
# df_ensemble_test = featutil.joinRunToEnsembleFrame(df_ensemble_test, subrunDir, fileName)

In [ ]:
df_ensemble_test.head()

In [ ]:
# Get all the columns
col_names = df_ensemble_test.columns

idCols = ['RowID']
feature_cols =  col_names.drop(['RowID'])

# Create the sub data sets of the features and the target
dfTestFeatures = df_ensemble_test[feature_cols]

In [ ]:
# load the ensemble model  and predict
model = lgb.Booster(model_file=modelsDir + filePrefix + "_ensemble_predictor.model")
predicted = model.predict(dfTestFeatures)

dfPredictions = df_ensemble_test[idCols]
dfPredictions["Score"] = predicted

# join the predictions to the ids, sort by rowid and write to out the subrun file
finalRunFilePath = runDir + filePrefix + "_run.tsv"
dfPredictions.to_csv(finalRunFilePath, sep="\t", index=False, header=False)


In [ ]:
# Clean up variables
del df_ensemble_test
del dfTestFeatures
del model
del predicted
del dfPredictions